Let us import all the required libraries
* We'll be using tensorflow and keras for our Deep Learning Networks.
* We're using numpy to work with arrays.
* Pandas to work with our data.
* The re library to work with text using regular expressions.

In [1]:
import tensorflow as tf
print(tf.__version__)
from tensorflow import set_random_seed
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, GRU, Dense, Dropout, SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle

1.14.0


Using TensorFlow backend.


In [0]:
import numpy as np
from numpy.random import seed
import pandas as pd
import re
from random import shuffle
import random

We'll be setting up random seeds to reproduce our results.

In [0]:
seed(1)
set_random_seed(2)
random.seed(3)

Since we were using colabs, we need to import our data from our drive.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Let's begin reading the lines.

In [0]:
lines_df = pd.read_csv('bible.txt', sep='\n', header=None, engine='python')
lines_df = lines_df.iloc[:,0]

Let us take a look at what we've got.

In [6]:
print(lines_df.iloc[3])

1:2 And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.


Now let us begin by cleaning the texts.
We'll be removing all punctuation, digits and special characters, keeping only the text and whitespace.


We'll also shuffle our dataset.

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[^A-Za-z\s]", "", text)
    return text

clean_lines = lines_df.apply(lambda x: clean_text(x))
print(clean_lines[0])
shuffle(clean_lines)
print(clean_lines[0])
print(f"Number of lines are {len(clean_lines)}")

the old testament of the king james version of the bible
 and their father said unto them what way went he for his sons had seen what way the man of god went which came from judah
Number of lines are 24608


Now we'll convert our text in sequences and then using the N-gram model convert it into a sequence of N-words.

An N-gram is a sequence of N words: a 2-gram (or bigram) is a two-word sequence of words like “look over”, “over there” and a 3-gram (or trigram) is a three-word sequence of words like “look over there”.

In [8]:
tokenizer = Tokenizer()

def get_sequence_tokens(text):
    
    tokenizer.fit_on_texts(text)
    total_words = len(tokenizer.word_index) + 1
    
    input_sequences = []
    for line in text:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

text_sequences, total_words = get_sequence_tokens(clean_lines[:1000])
print(text_sequences[:5])
print(tokenizer.sequences_to_texts(text_sequences[:5]))
print(f"Length of sequences array {len(text_sequences)}")

[[2, 37], [2, 37, 131], [2, 37, 131, 38], [2, 37, 131, 38, 11], [2, 37, 131, 38, 11, 21]]
['and their', 'and their father', 'and their father said', 'and their father said unto', 'and their father said unto them']
Length of sequences array 30440


As you can see from the example above, the sentence is converted into N number of words.

Now while feeding data, we require the number of inputs to be constant.

However every sentence will not have the same number of words in it.

Hence, inorder to adjust for this varying length we pad the sentences (pre or post).

The maximum sequence size is defined by the maximum length in the input_sequences.

In [9]:
def generate_padded_sequences(input_sequences):
    max_sequence_length = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences,
                                             maxlen=max_sequence_length,
                                             padding='pre'))
    
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    
    label = tf.keras.utils.to_categorical(label, num_classes=total_words)
    
    return predictors, label, max_sequence_length

predictors, label, max_sequence_length = generate_padded_sequences(text_sequences)
print(f"Total number of words: {total_words} | Maximum sequence length: {max_sequence_length}")
print(predictors[0])

Total number of words: 3245 | Maximum sequence length: 199
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 2]


We will now build the model using an Embedding layer and Stacked GRUs each with their own Dropout layer.

The mask_zero in the Embedding layer tells the model to ignore the padded zeros.

In [10]:
def build_model(max_sequence_length, total_words):
    input_len = max_sequence_length - 1
    model = Sequential()
    
    model.add(Embedding(total_words, 128, input_length=input_len, mask_zero=True))
    model.add(SpatialDropout1D(0.5))
    
    model.add(GRU(512, return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(GRU(512))
    model.add(Dropout(0.1))
    
    model.add(Dense(total_words, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = build_model(max_sequence_length, total_words)
model.summary()

W0818 14:08:49.915250 140590955173760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0818 14:08:49.920247 140590955173760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0818 14:08:49.931137 140590955173760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0818 14:08:49.956823 140590955173760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0818 14:08:49.966783 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 198, 128)          415360    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 198, 128)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 198, 512)          984576    
_________________________________________________________________
dropout_1 (Dropout)          (None, 198, 512)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 512)               1574400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3245)              1664685   
Total para

To start training, we fit the model to the training data.

In [12]:
# Training the model
model.fit(predictors, label, batch_size=128, epochs=20)

Epoch 1/20
30440/30440 [==============================] - 230s 8ms/step - loss: 6.2667 - acc: 0.0847
Epoch 2/20
30440/30440 [==============================] - 224s 7ms/step - loss: 5.5418 - acc: 0.1387
Epoch 3/20
30440/30440 [==============================] - 224s 7ms/step - loss: 5.0598 - acc: 0.1675
Epoch 4/20
30440/30440 [==============================] - 224s 7ms/step - loss: 4.6192 - acc: 0.1896
Epoch 5/20
30440/30440 [==============================] - 224s 7ms/step - loss: 4.1481 - acc: 0.2128
Epoch 6/20
30440/30440 [==============================] - 224s 7ms/step - loss: 3.6375 - acc: 0.2476
Epoch 7/20
30440/30440 [==============================] - 224s 7ms/step - loss: 3.1532 - acc: 0.3115
Epoch 8/20
30440/30440 [==============================] - 224s 7ms/step - loss: 2.7191 - acc: 0.3830
Epoch 9/20
30440/30440 [==============================] - 224s 7ms/step - loss: 2.3563 - acc: 0.4511
Epoch 10/20
30440/30440 [==============================] - 224s 7ms/step - loss: 2.0641 - a

Saving the model to use in the future.

In [0]:
model.save('model_text_gen.h5')

The generate_text method uses a start word or phrase that is used to predict the next probable word. This word is then added to the overall output and is fed to the model again to generate the next word. This keeps on continuing till we reach the desired number of words. 

As our model was trained on sequences of text, we perform the same conversion before every prediction.

In case we provide a word out of the model's vocabulary, we choose a random class from the top 10 classes the model predicts. We do this to avoid the same output sequence for every word not from its vocabulary list.

In [0]:
def generate_text(seed_text, num_words, model, max_sequence_length):
    for _ in range(num_words):
        generate_random = False
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        if len(token_list) == 0:
            generate_random = True
        token_list = pad_sequences([token_list],
                                   maxlen=max_sequence_length - 1,
                                             padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        if generate_random:
            pred_list = np.argsort(-model.predict_proba(token_list))[0]
            predicted = pred_list[np.random.randint(low=0, high=10)]
            print(predicted)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

Let's generate some text using the function we created above.

In [15]:
print(generate_text("he said", 15, model, max_sequence_length))

He Said Unto Them Go And See I Have Heard A Rumour With The Lord And Ye


Let's load our model as try predicting text using the new_model

In [16]:
new_model = tf.keras.models.load_model('model_text_gen.h5')
new_model.summary()

W0818 15:37:35.822065 140590955173760 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 198, 128)          415360    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 198, 128)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 198, 512)          984576    
_________________________________________________________________
dropout_1 (Dropout)          (None, 198, 512)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 512)               1574400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3245)             

In [17]:
print(generate_text("he said", 15, new_model, max_sequence_length))

He Said Unto Them Go And See I Have Heard A Rumour With The Lord And Ye


As you can see from the above output the new_model is exactly the same as the original model.

A few more examples below.

In [18]:
print(generate_text("death", 5, model, max_sequence_length))
print(generate_text("lord", 10, model, max_sequence_length))
print(generate_text("seek", 15, model, max_sequence_length))
print(generate_text("good", 20, model, max_sequence_length))
print(generate_text("silence", 25, model, max_sequence_length))

Death Is The Lord God Of
Lord The Eldest Day Of The Riches Both Of The Children
Seek The Lord God Of Hosts Shall Not Be Moved And The Effect Shall Be In
Good Thyself And The Law Is The Lord Who Is Not By The Bands Of The Lord The Rebuke Of Thy
Silence From The People And The Lord Left And Aaron And His Sons Shall Be Like Unto His Own Neighbour As A Generations Of The Lord


Saving the tokenizer for future use.

In [0]:
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
